In [1]:
# Testing the downloading data method from:
# https://claut.gitlab.io/man_ccia/lab2.html
# Import the required modules
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests
from tqdm import tqdm

# Set the os environment to on
os.environ['ESGF_PYCLIENT_NO_FACETS_STAR_WARNING'] = "on"

In [2]:
# Import the functions
from testing_download_functions import query_data_esgf, extract_file_context

In [3]:
# set the search connection
# to the LLNL search node
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search', distrib=True)

In [4]:
print(type(conn))

<class 'pyesgf.search.connection.SearchConnection'>


In [5]:
# test the function for querying the database
results = query_data_esgf(conn,
                        source_id='CanESM5',
                        experiment_id='hist-aer',
                        variable_id='tas',
                        table_id='Amon',
                        member_id='r1i1p1f1, r2i1p1f1',
                        data_node='crd-esgf-drc.ec.gc.ca',)

# print the len of the results
print(len(results))

# print the type of the results
print(type(results))

# print the results
print(results)

2
<class 'pyesgf.search.results.ResultSet'>


In [6]:
# Print the details of the first result
print(results[0].json['id'])

CMIP6.DAMIP.CCCma.CanESM5.hist-aer.r1i1p1f1.Amon.tas.gn.v20190429|crd-esgf-drc.ec.gc.ca


In [8]:
# Use the function to extract the file context
files_list = extract_file_context(results)

# print the files list
print(files_list)

Extracting file context for 2 datasets...
[[{'filename': 'tas_Amon_CanESM5_hist-aer_r1i1p1f1_gn_185001-202012.nc', 'url': 'http://crd-esgf-drc.ec.gc.ca/thredds/fileServer/esgE_dataroot/AR6/CMIP6/DAMIP/CCCma/CanESM5/hist-aer/r1i1p1f1/Amon/tas/gn/v20190429/tas_Amon_CanESM5_hist-aer_r1i1p1f1_gn_185001-202012.nc'}], [{'filename': 'tas_Amon_CanESM5_hist-aer_r2i1p1f1_gn_185001-202012.nc', 'url': 'http://crd-esgf-drc.ec.gc.ca/thredds/fileServer/esgE_dataroot/AR6/CMIP6/DAMIP/CCCma/CanESM5/hist-aer/r2i1p1f1/Amon/tas/gn/v20190429/tas_Amon_CanESM5_hist-aer_r2i1p1f1_gn_185001-202012.nc'}]]
